In [ ]:
import tkinter as tk

In [15]:
board_size = 8

# Inicializar los colores de las casillas
white_color = 'white'
black_color = 'black'
default_color = 'lightblue'
board = [[' ' for _ in range(board_size)] for _ in range(board_size)]
board[3][3] = board[4][4] = 'W'
board[3][4] = board[4][3] = 'B'

In [14]:
points_direction = {
    0: 'N',
    1: 'NE',
    2: 'E',
    3: 'SE',
    4: 'S',
    5: 'SO',
    6: 'O',
    7: 'NO'
}
directions = [(-1, 0), (-1, 1), (0, 1), (1, 1), (1, 0), (1, -1), (0, -1), (-1, -1)]

def check_mov_direction(row, col):
    pos=0
    direction_check=[]
    check =False
    for dr, dc in directions:
        if(board[row+dr][col+dc]=='B'):
            direction_check.append(points_direction[pos])
            check= True
        pos+=1       
    return direction_check,check
def check_mov(row, col,direction_check, player):
    coor_repaint=[]
    for dir in direction_check:
        x,y=row, col
        if dir == "N":
            while board[y][x] != player and  board[y][x] != ' ':
               coor_repaint.append(x,y) 
               y-=1
               x+=1 
        elif dir == "NE":
            while board[y][x] != player and  board[y][x] != ' ':
               coor_repaint.append(x,y) 
               y-=1
               x+=1  
        elif dir == "E":
            while board[y][x] != player and  board[y][x] != ' ':
               coor_repaint.append(x,y) 
               x+=1 
        elif dir == "SE":
            while board[y][x] != player and  board[y][x] != ' ':
               coor_repaint.append(x,y) 
               y+=1
               x+=1 
        elif dir == "S":
            while board[y][x] != player and  board[y][x] != ' ':
               coor_repaint.append(x,y) 
               y+=1
                
        elif dir == "SO":
            while board[y][x] != player and  board[y][x] != ' ':
               coor_repaint.append(x,y) 
               y+=1
               x-=1 
        elif dir == "O":
            while board[y][x] != player and  board[y][x] != ' ':
               coor_repaint.append(x,y) 
               x-=1
        elif dir == "NO":
            while board[y][x] != player and  board[y][x] != ' ':
               coor_repaint.append(x,y) 
               y-=1
               x-=1 
        else:
            pass 
            
    return coor_repaint

In [18]:


# Tamaño del tablero (8x8)

# Función para crear una casilla en el tablero
def crear_casilla(tablero, row, col, color=default_color):
    button = tk.Button(tablero, text='', command=lambda r=row, c=col: pintar_casilla(r, c), width=3, bg=color)
    button.grid(row=row + 1, column=col + 1)
    return button

# Función para pintar una casilla
def pintar_casilla(row, col):
    if board[row][col] == ' ':
        direction_check,check= check_mov_direction(row, col)
        if check:
            color = white_color if current_color == 'W' else black_color
            coor_repaint= check_mov(row, col,direction_check, color)
            for y,x in coor_repaint:
                print(coor_repaint)
                break
                board[y][x] = current_color
                crear_casilla(tablero, y, x, color)
                cambiar_color()
            imprimir_coordenadas(row, col)  
# Función para cambiar el color actual
def cambiar_color():
    global current_color
    current_color = 'W' if current_color == 'B' else 'B'
    label_color.config(text=f'Color actual: {current_color}')

def imprimir_coordenadas(row, col):
    print(f"Posición seleccionada: ({row}, {col})")

root = tk.Tk()
root.title('Othello')

tablero = tk.Frame(root)
tablero.pack()

current_color = 'B'

label_color = tk.Label(root, text=f'Color actual: {current_color}')
label_color.pack()

def darw_map():
    for i in range(board_size):
        tk.Label(tablero, text=str(i + 1), width=3).grid(row=i + 1, column=0)
        tk.Label(tablero, text=chr(ord('A') + i), width=3).grid(row=0, column=i + 1)

    # Crear los botones del tablero
    for row in range(board_size):
        for col in range(board_size):
            if (row == 3 and col == 3) or (row == 4 and col == 4):
                crear_casilla(tablero, row, col, 'white')
            elif (row == 3 and col == 4) or (row == 4 and col == 3):
                crear_casilla(tablero, row, col, 'black')
            else:
                crear_casilla(tablero, row, col)

darw_map()
root.mainloop()


Posición seleccionada: (3, 2)
Posición seleccionada: (3, 2)


In [12]:
board

[['B', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
 [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
 [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
 [' ', ' ', ' ', 'W', 'B', ' ', ' ', ' '],
 [' ', ' ', ' ', 'B', 'W', ' ', ' ', ' '],
 [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
 [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
 [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']]